In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import nibabel as nib

from connectivity import constants as const
from connectivity import visualize_summary as summary

%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [10]:
# run connectivity routine
# train and validate ridge model(s)
# evaluate best model (based on CV R)

# navigate to scripts dir
dirs = const.Dirs()
os.chdir(os.path.join(Path(dirs.base_dir).parent, 'connectivity/scripts'))

## train 
# %run -i 'script_mk.py' --cortex="tesselsWB162" --model_type="ridge" --train_or_eval="train"

## eval
# %run -i 'script_mk.py' --train_or_eval=="eval"

In [ ]:
# plot alpha curve for train CVs

df = summary.train_summary()

summary.plot_train_predictions(dataframe=tmp, hue='train_exp');


In [ ]:
# Model evaluation (noise ceiling and predictions)

# get evaluation 
df_eval = summary.eval_summary()

for exp in ['sc1', 'sc2']:
    
    #plot evaluation
    summary.plot_eval_predictions(dataframe=df_eval, exp=exp)


In [ ]:
# visualize voxel maps - evaluation
view = summary.plot_map(gifti_func='group_R_vox', exp='sc1', cscale=[0, 0.8])

view

In [ ]:
# visualize voxel maps - training

# view = summary.plot_train_map(gifti_func='group_weights_cerebellum', exp='sc1')

# view